In [15]:
import models
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [16]:
data = pd.read_csv("us_change_cleaned.csv")
X = data.drop(columns='Consumption')
y = data['Consumption']

In [17]:
X.shape, y.shape

((198, 5), (198,))

In [18]:
X_train = X[:int(0.8*len(X))]
y_train = y[:int(0.8*len(y))]
X_test = X[int(0.8*len(X)):]
y_test = y[int(0.8*len(y)):]
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((158, 5), (158,), (40, 5), (40,))

In [19]:
metrics = ['mse', 'mae']

In [20]:
# Mean 
model = models.Mean()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.evaluate(metrics=metrics, X=X_test, y=y_test)


{'mse': 0.12819038773370228, 'mae': 0.2858484325156872}

In [21]:
print(y_train.mean())
print(y_pred)

0.7782184182306155
[0.77821842 0.77821842 0.77821842 0.77821842 0.77821842 0.77821842
 0.77821842 0.77821842 0.77821842 0.77821842 0.77821842 0.77821842
 0.77821842 0.77821842 0.77821842 0.77821842 0.77821842 0.77821842
 0.77821842 0.77821842 0.77821842 0.77821842 0.77821842 0.77821842
 0.77821842 0.77821842 0.77821842 0.77821842 0.77821842 0.77821842
 0.77821842 0.77821842 0.77821842 0.77821842 0.77821842 0.77821842
 0.77821842 0.77821842 0.77821842 0.77821842]


In [22]:
# RandomWalk
model = models.RandomWalk()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.evaluate(metrics=metrics, X=X_test, y=y_test)


{'mse': 1.1112844464118405, 'mae': 1.0071672410592745}

In [23]:
print(y_train.iloc[-1])
print(y_pred)

-0.405843686476004
[-0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369
 -0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369
 -0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369
 -0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369
 -0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369
 -0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369 -0.40584369
 -0.40584369 -0.40584369 -0.40584369 -0.40584369]


In [24]:
# LinearRegression
model = models.LinearRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
model.evaluate(metrics=metrics, X=X_test, y=y_test)

{'mse': 0.02238550411797885, 'mae': 0.11513144763974555}